In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
filename='2021_LoL_esports_match_data_from_OraclesElixir_20221108.csv'
df=pd.read_csv(filename)

df.head()



,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,118.0,748.0,-56.0,-4.0,2.0,0.0,1.0,0.0,1.0,1.0
1,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,84.0,2120.0,3405.0,62.0,3.0,2.0,0.0,2.0,0.0,3.0
2,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,143.0,1578.0,354.0,15.0,2.0,3.0,0.0,0.0,0.0,0.0
3,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,120.0,124.0,102.0,10.0,0.0,4.0,2.0,1.0,1.0,2.0
4,ESPORTSTMNT03/1632489,complete,http://matchhistory.na.leagueoflegends.com/en/...,KeSPA,2021,NaN,0,2021-01-02 07:40:39,1.0,10.25,...,25.0,448.0,450.0,3.0,1.0,4.0,0.0,0.0,2.0,2.0


In [3]:
#Checking for any missing data type and values

df.info()
num_vars = df.columns[df.dtypes != 'object']
cat_vars = df.columns[df.dtypes == 'object']

print(num_vars)
print(cat_vars)
df[num_vars].isnull().sum().sort_values(ascending =False)/len(df)

#checking if the columns that are relatable to my hypothesis are as bad in missing data
print(df['xpdiffat15'].isnull().sum()/len(df))
print(df['golddiffat15'].isnull().sum()/len(df))
print(df['killsat15'].isnull().sum()/len(df))
print(df['csdiffat15'].isnull().sum()/len(df))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150264 entries, 0 to 150263
Columns: 123 entries, gameid to opp_deathsat15
dtypes: float64(94), int64(11), object(18)
memory usage: 141.0+ MB
Index(['year', 'playoffs', 'game', 'patch', 'participantid', 'gamelength',
       'result', 'kills', 'deaths', 'assists',
       ...
       'opp_csat15', 'golddiffat15', 'xpdiffat15', 'csdiffat15', 'killsat15',
       'assistsat15', 'deathsat15', 'opp_killsat15', 'opp_assistsat15',
       'opp_deathsat15'],
      dtype='object', length=105)
Index(['gameid', 'datacompleteness', 'url', 'league', 'split', 'date', 'side',
       'position', 'playername', 'playerid', 'teamname', 'teamid', 'champion',
       'ban1', 'ban2', 'ban3', 'ban4', 'ban5'],
      dtype='object')
0.08592876537294362
0.08592876537294362
0.08592876537294362
0.08592876537294362


In [11]:
cols=['gameid','datacompleteness','url','league','split','date',
        'side','position','playername','playerid','teamname','teamid',
        'champion','ban1','ban2','ban2','ban3','ban4','ban5','year','playoffs','game',
        'patch','participantid','gamelength','result','kills','deaths','assists','doublekills'
        ,'triplekills','quadrakills','pentakills','firstblood','firstbloodassist',
        'firstbloodvictim','teamkills','teamdeaths','firstbloodkill','team kpm','ckpm','firstdragon',
        'dragons','opp_dragons','elementaldrakes','opp_elementaldrakes','infernals',
     'mountains','clouds','oceans','chemtechs','hextechs','dragons (type unknown)',
     'elders','opp_elders','firstherald','']
df.drop(cols,axis=1)

,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,118.0,748.0,-56.0,-4.0,2.0,0.0,1.0,0.0,1.0,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,84.0,2120.0,3405.0,62.0,3.0,2.0,0.0,2.0,0.0,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,143.0,1578.0,354.0,15.0,2.0,3.0,0.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,120.0,124.0,102.0,10.0,0.0,4.0,2.0,1.0,1.0,2.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.0,448.0,450.0,3.0,1.0,4.0,0.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150259,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,142.0,-108.0,-322.0,-15.0,1.0,0.0,1.0,0.0,2.0,0.0
150260,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,124.0,2148.0,848.0,25.0,2.0,0.0,0.0,1.0,1.0,2.0
150261,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,22.0,-99.0,-214.0,0.0,0.0,2.0,2.0,1.0,1.0,1.0
150262,2.0,0.0,0.0,1.0,1.0,0.0,5.0,9.0,0.0,0.0,...,529.0,127.0,780.0,8.0,5.0,7.0,4.0,4.0,4.0,5.0


In [ ]:
#to_drop10=['redWins','redTotalGolds','redCurrentGolds','redTotalLevel','redAvgLevel',
      #     'redTotalMinionKills','redTotalJungleMinionKills','redFirstBlood','redKill']